In [1]:
from operator import add

In [6]:
f = sc.textFile('README.md')
wc = f.flatMap(lambda x: x.split(' ')).map(lambda x: (x, 1)).reduceByKey(add).cache()
wc.collect()

[(u'', 67),
 (u'when', 1),
 (u'R,', 1),
 (u'including', 3),
 (u'computation', 1),
 (u'using:', 1),
 (u'guidance', 2),
 (u'Scala,', 1),
 (u'environment', 1),
 (u'only', 1),
 (u'rich', 1),
 (u'Apache', 1),
 (u'sc.parallelize(range(1000)).count()', 1),
 (u'Building', 1),
 (u'guide,', 1),
 (u'return', 2),
 (u'Please', 3),
 (u'Try', 1),
 (u'not', 1),
 (u'Spark', 13),
 (u'scala>', 1),
 (u'Note', 1),
 (u'cluster.', 1),
 (u'./bin/pyspark', 1),
 (u'params', 1),
 (u'through', 1),
 (u'GraphX', 1),
 (u'[run', 1),
 (u'abbreviated', 1),
 (u'[project', 2),
 (u'##', 8),
 (u'library', 1),
 (u'see', 1),
 (u'"local"', 1),
 (u'[Apache', 1),
 (u'will', 1),
 (u'#', 1),
 (u'processing,', 1),
 (u'for', 11),
 (u'[building', 1),
 (u'provides', 1),
 (u'print', 1),
 (u'supports', 2),
 (u'built,', 1),
 (u'[params]`.', 1),
 (u'available', 1),
 (u'run', 7),
 (u'tests](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools).',
  1),
 (u'This', 2),
 (u'Hadoop,', 2),
 (u'Tests', 1),
 (u'example:', 1),

In [16]:
import heapq

In [17]:
def mapper(counts):
    yield (0, heapq.nlargest(3, counts, lambda pair: pair[1]))

def reducer(top1, top2):
    return heapq.nlargest(3, top1+top2, lambda pair: pair[1])

topWords = wc.mapPartitions(mapper).reduceByKey(reducer)
topWords.collect()

[(0, [(u'', 67), (u'the', 21), (u'to', 14)])]

In [20]:
def mapper(counts):
    yield heapq.nlargest(3, counts, lambda pair: pair[1])

def reducer(top1, top2):
    return heapq.nlargest(3, top1+top2, lambda pair: pair[1])

topWords = wc.mapPartitions(mapper).reduce(reducer)
topWords

[(u'', 67), (u'the', 21), (u'to', 14)]

In [21]:
trips = sc.textFile('citibike.csv').cache()

In [35]:
def mapper(lines):
    bins = {}
    for line in lines:
        fields = line.split(',')
        minutes = int(fields[2])/60
        bins[minutes] = bins.get(minutes, 0) + 1
    yield (0, bins)
    
def reducer(bins1, bins2):
    bins = bins1
    for (m,c) in bins2.iteritems():
        bins[m] = bins.get(m, 0) + c
    return bins

def findMedian((key,bins)):
    total = sum(bins.itervalues())
    median = 0
    for (m, c) in bins.iteritems():
        median += c
        if median>total/2:
            return m
    return 0

median = trips.mapPartitions(mapper).reduceByKey(reducer).map(findMedian)
median.collect()

[8]

In [39]:
def mapper(lines):
    bins = {}
    for line in lines:
        fields = line.split(',')
        minutes = int(fields[2])/60
        bins[minutes] = bins.get(minutes, 0) + 1
    yield (0, bins)
    
def reducer((bins, total), bins2):
    for (m,c) in bins2.iteritems():
        bins[m] = bins.get(m, 0) + c
        total += c
    return (bins, total)

def findMedian((key,binsList)):
    (bins, total) = reduce(reducer, binsList, ({},0))
    median = 0
    for (m, c) in bins.iteritems():
        median += c
        if median>total/2:
            return m
    return 0

median = trips.mapPartitions(mapper).groupByKey().map(findMedian)
median.collect()

[8]